In [203]:
import re 
import os

# wstępne czyszczenie pliku: stosunkowo łatwiej, szybciej i bardziej czytelnie było mi dokonać czyszczenia pliku txt
# przed importem do dataframe - nieregularności w separacji wartości okazały się znacznie większym problemem niż
# przypuszczałem, dlatego też usunąłem wszystkie nieścisłości, a następnie zapisałem zedytowany plik txt do folderu
# Analysis Data

with open('../Original Data/weather.txt') as f:
    lines = f.readlines()

lines_edit = []
regex_1 = re.compile('(\s+I\s*|\s+S\s*|\s+)')
# rozwiązanie z poniższym regexem nie jest optymalne (ucina M oraz N lub też P i zamienia na Q) lecz będzie działać jeżeli o tym 
# fakcie nie zapomnimy podczas obróbki danych
regex_2 = re.compile('\w-9999\s')

for line in lines:
    line_without_sep = regex_1.sub(' ', line)
    line_seperated_nan = regex_2.sub('Q-9999 ', line_without_sep) + '\n'
    lines_edit.append(line_seperated_nan)

with open('../Analysis Data/weather_whitespace.txt', 'w') as f:
    f.writelines(lines_edit)


In [215]:
import pandas as pd 


my_cols = [str(i) for i in range(33)]
weather_df = pd.read_csv("../Analysis Data/weather_whitespace.txt",
                         sep=' ', 
                         header=None, 
                         names=my_cols)
weather_df.drop('32', axis=1, inplace=True)
weather_df.head()

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,MX000017004195504TMAX,310,310,310,320,330,320,320,330,330,...,330,330,330,330,330,330,340,330,320,-9999
1,MX000017004195504TMIN,150,150,160,150,160,160,160,160,160,...,170,170,170,180,190,190,170,180,160,-9999
2,MX000017004195504PRCP,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,6,-9999
3,MX000017004195505TMAX,310,310,310,300,300,300,310,310,310,...,330,340,350,330,310,310,320,310,300,290
4,MX000017004195505TMIN,200,160,160,150,150,150,160,160,170,...,170,190,190,190,180,160,150,170,150,160


In [216]:
weather_df.shape

(1714, 32)

In [217]:
# teraz kolumne 0 trzeba podzielić na 3 człony: nazwa stacji badawczej(MX000017004) (wszędzie będzie taka sama, 
# dlatego też można się jej pozbyć zupełnie), rok-miesiąc(1955-09), nazwa danej (np. TMAX)

# początek tego wszystkiego jest zawsze taki sam - można podzielić po 11, 17 literach
# weather_edit['Stacja'] = weather_edit[0].str[0:11]
weather_df['Data'] = weather_df['0'].str[11:17].astype(int)
weather_df['Dana'] = weather_df['0'].str[17:(17+3)]
weather_df.drop(labels=['0'], axis=1, inplace=True)
weather_df.tail()

,1,2,3,4,5,6,7,8,9,10,...,24,25,26,27,28,29,30,31,Data,Dana
1709,-9999,-9999,-9999,148,-9999,-9999,-9999,-9999,127,-9999,...,-9999,162,-9999,-9999,-9999,-9999,170,NaN,201103,TMI
1710,0,-9999,0,0,0,-9999,0,0,0,0,...,0,-9999,0,-9999,-9999,0,-9999,0,201103,PRC
1711,350,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,...,-9999,-9999,-9999,-9999,-9999,-9999,-9999,NaN,201104,TMA
1712,168,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,...,-9999,-9999,-9999,-9999,-9999,-9999,-9999,NaN,201104,TMI
1713,0,0,-9999,-9999,0,0,-9999,-9999,-9999,-9999,...,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,201104,PRC


In [218]:
# teraz robimy tak, żeby każdy dzień był osobnym rekordem
weather_melted = weather_df.melt(id_vars=['Data', 'Dana'])
weather_melted.drop(labels=['variable'], axis=1, inplace=True)
weather_melted.head()

,Data,Dana,value
0,195504,TMA,310
1,195504,TMI,150
2,195504,PRC,0
3,195505,TMA,310
4,195505,TMI,200


In [234]:
# zamieniamy tak, żeby TMAX, TMIN oraz PRCP były osobnymi kolumnami
weather_columns = weather_melted['Data'][weather_melted['Dana'] == 'TMA'].reset_index()
weather_columns.drop('index', axis=1, inplace=True)

mask_tmax = weather_melted['Dana'] == 'TMA'
mask_tmin = weather_melted['Dana'] == 'TMI'
mask_prcp = weather_melted['Dana'] == 'PRC'

tmax = weather_melted['value'][mask_tmax].reset_index()
tmax.drop('index', axis=1, inplace=True)
weather_columns['TMAX'] = tmax

tmin = weather_melted['value'][mask_tmin].reset_index()
tmin.drop('index', axis=1, inplace=True)
weather_columns['TMIN'] = tmin

prcp = weather_melted['value'][mask_prcp].reset_index()
prcp.drop('index', axis=1, inplace=True)
weather_columns['PRCP'] = prcp

weather_columns.head()

,Data,TMAX,TMIN,PRCP
0,195504,310,150,0
1,195505,310,200,0
2,195506,300,160,103
3,195507,270,150,18
4,195508,230,140,24


In [260]:
# odcinam koniec danych - jest on na tyle wybrakowany że nie ma sensu go analizowac 
weather_columns_cut = weather_columns.iloc[:17000,:]

# zapisuje uzyskany wynik do pliku csv w folderze analysed data
weather_columns_cut.to_csv('../Analysis Data/weather_records.txt')

Podsumowując: dataset został przekształcony w taki sposób, że każdy dzień jest osobnym rekordem, a każda zmienna (TMAX, TMIN, PRCP) jest osobną kolumną. Takie przekształcenie znacząco ułatwia obchodzenie się z oraz analizę danych. Jednakowoż obecnie zbiór danych ma jedną dużą wadę - jest znacząco wybrakowany. Mam wrażenie że może to być spowodowane błędami w mojej obróbce, lecz nie jestem w stanie ich znaleźć pomimo wielu prób. Robiłem już dosyć sporo w zakresie czyszczenia danych w pandasie, lecz to jest coś czemu nie życzę najgorszemu wrogowi - jednocześnie chciałem zaznaczyć że poziom trudności tego zbioru danych był nieporównywalnie większy z innymi (np jeden już zupełnie gotowy do zwyczajnej analizy bez czyszczenia). Dodatkowo już na stronie laboratorium jest on źle opisany (to nie są dane z 2010 roku, lecz z wielu lat) co tylko utrudnia pracę.